<a href="https://colab.research.google.com/github/ayachiii/soturon/blob/main/export_HRV_parameters%E8%87%AA%E4%BD%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ECG_人名_フェーズ.csvから、人名_HRV.csvを出力するコード

In [5]:
!pip install heartpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.5 MB/s eta 0:00:00


In [6]:
import heartpy as hp
import pandas as pd
import os

ECG_人名_フェーズ.csvの５つのファイルをアップロードする。(boredom,flow,flow_ultra,overload,rest)

In [9]:
# 心拍変動パラメータの計算

SAMPLING_RATE = 500.0 # サンプリング周波数 (500Hzと仮定)

PHASES_TO_ANALYZE = ["rest", "boredom", "flow", "flow_ultra", "overload"]
SUBJECT_NAME = "kumakura" # 被験者名ここは手作業で自分で変えるのかも...
OUTPUT_HRV_FILE = f"{SUBJECT_NAME}_HRV.csv" # 最終的な出力ファイル名

TARGET_COLUMNS = [
    'bpm', 'ibi', 'sdnn', 'sdsd', 'rmssd', 'pnn20', 'pnn50', 'hr_mad',
    'sd1', 'sd2', 's', 'sd1/sd2', 'breathingrate', 'vlf', 'lf', 'hf', 'lf/hf',
    'p_total', 'vlf_perc', 'lf_perc', 'hf_perc', 'lf_nu', 'hf_nu',
    'min_lf', 'min_hf', 'lf_diff', 'hf_diff', 'lfhf_diff'
]
#初期値を設定。初めは０にしておく。restフェーズ解析完了時には値が更新されている。
lf_rest = 0.0
hf_rest = 0.0
lfhf_rest = 0.0


for phase_name in PHASES_TO_ANALYZE:
    INPUT_ECG_FILE = f"ECG_{SUBJECT_NAME}_{phase_name}.csv"
    print(f"\n--- 処理開始: {phase_name} フェーズ ---")

# --- 1. ECGデータの読み込み ---
    try:
    # column_name="ECG" は、ECGデータが入っている列名を指定
      hrdata = hp.get_data(INPUT_ECG_FILE, delim=",", column_name="ECG")
    except Exception as e:
      print(f"エラー: {INPUT_ECG_FILE} が見つからないか、読み込めません。スキップします。")
      continue

# --- 2. HRVパラメータの計算 (元のコードの再現) ---
    START_TIME_SEC = 60 # 解析開始時間を60秒に設定
    START_INDEX = int(START_TIME_SEC * SAMPLING_RATE)

# 最初の60秒間を除外したデータで処理を実行
    try:
        working_data, measures = hp.process(
          hrdata[START_INDEX:],
          SAMPLING_RATE,
          report_time=True,
          calc_freq=True
        )
    except Exception as e:
        print(f"HeartPy解析エラー: {e}")
        continue


    print("LF/HFの最小値をスキャン中...")
    current_time = int(120 * SAMPLING_RATE) # 120秒から開始
    min_lf = 999999
    min_hf = 999999
    slice_value_s = 10

    while current_time < len(hrdata):
        try:
            # 0から現在時刻までのスライスで解析
            hrdata_slice = hrdata[0: current_time]
            # calc_freq=TrueだけでOK (report_timeはうるさいのでFalse)
            wd_slice, m_slice = hp.process(hrdata_slice, SAMPLING_RATE, report_time=False, calc_freq=True)

            min_lf = min(min_lf, m_slice["lf"])
            min_hf = min(min_hf, m_slice["hf"])
        except:
            pass # 短すぎてエラーが出る場合などは無視して進める

        current_time += int(slice_value_s * SAMPLING_RATE)

    # ループが終わったら結果を格納 (初期値のままなら0を入れる)
    measures["min_lf"] = min_lf if min_lf != 999999 else 0.0
    measures["min_hf"] = min_hf if min_hf != 999999 else 0.0


    # --- 4. pnn20/50 の単位統一 ---
    # ★重要: 分岐の前に計算することで、Restもそれ以外も同じ単位にします。
    # もし目標ファイル(0.19...)に合わせるなら、以下の2行はコメントアウトしてください。
    # パーセンテージ(19.0%)にしたいなら、このまま有効にしてください。
    #measures['pnn20'] = measures.get('pnn20', 0.0) * 100
    #measures['pnn50'] = measures.get('pnn50', 0.0) * 100


# --- 5. 差分計算と保存 ---
    if phase_name == "rest":
        # 基準値を保存
        lf_rest = measures["lf"]
        hf_rest = measures["hf"]
        lfhf_rest = measures["lf/hf"]

        measures["lf_diff"] = 0
        measures["hf_diff"] = 0
        measures["lfhf_diff"] = 0

    # 新規保存（ヘッダーあり）
        df = pd.DataFrame(measures, index=[f"{phase_name}"])
        df=df.reindex(columns=TARGET_COLUMNS)
        df.to_csv(OUTPUT_HRV_FILE, sep=',', header=True)
        print(f"Rest結果を新規保存しました。")

    else:
        # 差分計算
        measures["lf_diff"] = lf_rest - measures["lf"]
        measures["hf_diff"] = hf_rest - measures["hf"]
        measures["lfhf_diff"] = lfhf_rest - measures["lf/hf"]

        # 追記保存（ヘッダーなし）
        df = pd.DataFrame(measures, index=[f"{phase_name}"])
        df.to_csv(OUTPUT_HRV_FILE, sep=',', mode="a", header=False)
        print(f"{phase_name} 結果を追記しました。")

print(f"\n--- 全フェーズ処理完了 ---")


--- 処理開始: rest フェーズ ---

Finished in 0.221013 sec
LF/HFの最小値をスキャン中...
Rest結果を新規保存しました。

--- 処理開始: boredom フェーズ ---

Finished in 0.165013 sec
LF/HFの最小値をスキャン中...
boredom 結果を追記しました。

--- 処理開始: flow フェーズ ---

Finished in 0.475441 sec
LF/HFの最小値をスキャン中...
flow 結果を追記しました。

--- 処理開始: flow_ultra フェーズ ---

Finished in 0.128821 sec
LF/HFの最小値をスキャン中...
flow_ultra 結果を追記しました。

--- 処理開始: overload フェーズ ---

Finished in 0.175717 sec
LF/HFの最小値をスキャン中...
overload 結果を追記しました。

--- 全フェーズ処理完了 ---
